In [1]:
import torch
import numpy as np
import random
import os

from torch import nn

In [2]:
# You may replace the workspace directory if you want.
workspace_dir = '.'

In [3]:
def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2022)

In [6]:
from torch.utils.data import Dataset, DataLoader

class LMDBDataset(Dataset):
    def __init__(self, fnames, transform):
        self.transform = transform
        self.fnames = fnames
        self.num_samples = len(self.fnames)

    def __getitem__(self,idx):
        fname = self.fnames[idx]
        # 1. Load the image
        img = torchvision.io.read_image(fname)
        # 2. Resize and normalize the images using torchvision.
        img = self.transform(img)
        return img

    def __len__(self):
        return self.num_samples
    
def get_dataset(root):
    fnames = glob.glob(os.path.join(root, '*'))
    # 1. Resize the image to (64, 64)
    # 2. Linearly map [0, 1] to [-1, 1]
    compose = [
        transforms.ToPILImage(),
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),
    ]
    transform = transforms.Compose(compose)
    dataset = LMDBDataset(fnames, transform)
    return dataset


In [ ]:
dataset = get_dataset(os.path.join(workspace_dir, 'faces'))

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

# Define model
class CRNN(nn.Module):
    """
    Input shape: (N, 1, h, w)  # W * 32 gray-scale image
    Output shape: (N, )
    """
    def __init__(self, nclass):
        super(CRNN, self).__init__()
        # self.flatten = nn.Flatten()
        self.cnn_layer = nn.Sequential(
            nn.Conv2d(1, 64, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            
            nn.Conv2d(64, 128, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2, 2), stride=2),
            
            nn.Conv2d(128, 256, 3, stride=1, padding=1),
            
            nn.Conv2d(256, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((1, 2), stride=2),
            
            nn.Conv2d(256, 512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            
            nn.Conv2d(512, 512, 3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d((1, 2), stride=2),
            
            nn.Conv2d(512, 512, 2, stride=1, padding=0)
        )
        
        # Input shape: (L, N, Input_size)
        # Output shape:(L, N, D*proj_size)
        self.BiLSTM = nn.Sequential(
            nn.LSTM(512, 256, bidirectional=True),
            nn.LSTM(256, 256, bidirectional=True, proj_size=nclass)
        )
        
        self.apply(weights_init)

    def forward(self, x):
        # conv features
        y_cnn = self.cnn_layer(x)
        b, c, h, w = y_cnn.size()
        assert h == 1, "the height of conv must be 1"
        y_cnn = y_cnn.squeeze(2)
        y_cnn = y_cnn.permute(2, 0, 1)  # [w, b, c]
        
        # rnn features
        logits = self.BiLSTM(y_cnn)  # [L, N, Input_size]
        return logits

# model = CRNN(nc).to(device)
# print(model)

Using cuda device


In [5]:
nc = 36

# Training hyperparameters
batch_size = 64
lr = 2e-4
n_epoch = 100

log_dir = os.path.join(workspace_dir, 'logs', 'test', 'base')
ckpt_dir = os.path.join(workspace_dir, 'checkpoints', 'test', 'base')
os.makedirs(log_dir, exist_ok=True)
os.makedirs(ckpt_dir, exist_ok=True)

# Model
crnn = CRNN(nc).to(device)
crnn.train()

# Loss
ctc_loss = nn.CTCLoss()

# Optimizer
opt = torch.optim.Adam(crnn.parameters(), lr=lr, betas=(0.5, 0.999))

# DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, 
                        shuffle=True, num_workers=4)

NameError: name 'DataLoader' is not defined